# Action_Clf - Multinomial Naive Bayes

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
import joblib

# Load Data

In [2]:
df = pd.read_parquet(r"input_data\ccd_labeled_train_validate.parquet")

print(df.shape)
display(df.head())                    

(13268, 6)


date              dlg_id  \
0  2024-10-05  2024-10-07_1017--0   
1  2024-10-05  2024-10-07_1017--1   
2  2024-10-05  2024-10-07_1017--2   
3  2024-10-05  2024-10-07_1017--3   
4  2024-10-05  2024-10-07_1017--4   

                                                text product   action  \
0                      i am still waiting on my card    card  inquire   
1  what can i do if my card still hasnt arrived a...    card  inquire   
2  i have been waiting over a week is the card st...    card  inquire   
3  can i track my card while it is in the process...    card  inquire   
4  how do i know if i will get my card or if it i...    card  inquire   

  sentiment  
0  negative  
1  negative  
2  negative  
3   neutral  
4   neutral

In [3]:
# testing - check for duplicates or errors in labels
sorted(df['action'].unique())

['access',
 'activate',
 'apply',
 'block',
 'cancel',
 'close',
 'deposit',
 'dispute',
 'earn',
 'exchange',
 'find',
 'inquire',
 'link',
 'open',
 'pay',
 'receive',
 'redeem',
 'refund',
 'renew',
 'report',
 'reset',
 'retrieve',
 'schedule',
 'select',
 'transfer',
 'unblock',
 'unknown',
 'unlink',
 'update',
 'verify',
 'withdraw']

In [4]:
df['action'].value_counts()

action
inquire     3162
dispute     1009
transfer     662
deposit      644
apply        644
withdraw     570
pay          425
activate     423
report       420
verify       401
unknown      387
link         339
open         338
block        325
update       313
close        285
unblock      264
exchange     239
reset        236
cancel       232
redeem       210
renew        208
access       201
schedule     199
find         185
select       176
retrieve     166
earn         164
refund       151
unlink       147
receive      143
Name: count, dtype: int64

In [5]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['action'], test_size=0.2, random_state=42, stratify=df['action'])

In [6]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),  # Feature extraction using TF-IDF
    ('classifier', MultinomialNB())  # Classifier (Naive Bayes)
])
pipeline.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('classifier', MultinomialNB())])

In [7]:
# Make predictions and evaluate the model
y_pred = pipeline.predict(X_test)

# Print evaluation metrics
print("Accuracy:", round(accuracy_score(y_test, y_pred), 3))
print(classification_report(y_test, y_pred))

Accuracy: 0.306
              precision    recall  f1-score   support

      access       0.00      0.00      0.00        40
    activate       0.88      0.08      0.15        85
       apply       0.42      0.10      0.16       129
       block       0.43      0.05      0.08        65
      cancel       0.00      0.00      0.00        46
       close       1.00      0.14      0.25        57
     deposit       0.82      0.07      0.13       129
     dispute       0.70      0.44      0.54       202
        earn       0.00      0.00      0.00        33
    exchange       0.00      0.00      0.00        48
        find       0.00      0.00      0.00        37
     inquire       0.26      0.99      0.42       632
        link       1.00      0.01      0.03        68
        open       0.56      0.07      0.13        68
         pay       0.00      0.00      0.00        85
     receive       0.00      0.00      0.00        29
      redeem       0.00      0.00      0.00        42
      refun

C:\py312_env_IssPlp\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\py312_env_IssPlp\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\py312_env_IssPlp\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Predict with Trained Model

In [8]:
# Example usage of the trained model to make predictions on new data
new_texts = [
    "my card is stuck in the atm machine. I want it back.", 
    "Investing in stocks is risky. I want open a fixed deposit account.",
    "I want to pay off my loan."
]
predictions = pipeline.predict(new_texts)

for text, prediction in zip(new_texts, predictions):
    print(f'Text: "{text}" => Predicted Keyword: "{prediction}"')

Text: "my card is stuck in the atm machine. I want it back." => Predicted Keyword: "inquire"
Text: "Investing in stocks is risky. I want open a fixed deposit account." => Predicted Keyword: "deposit"
Text: "I want to pay off my loan." => Predicted Keyword: "inquire"


# Save Model

In [9]:
# Save the model using joblib
joblib.dump(pipeline, 'trained_model/action_clr_MNB_model.pkl')

print("Model saved!")

Model saved!


# Reload Model and do Predict

In [10]:
del pipeline

In [11]:
# Load the model from the file
pipeline = joblib.load('trained_model/action_clr_MNB_model.pkl')

In [12]:
new_texts = [
    "my card is stuck in the atm machine. I want it back.", 
    "Investing in stocks is risky.",
    "Why are your fixed deposit rates so terrible? I don't want to roll over my fixed deposit."
]
predictions = pipeline.predict(new_texts)

for text, prediction in zip(new_texts, predictions):
    print(f'Text: "{text}" => Predicted Keyword: "{prediction}"')

Text: "my card is stuck in the atm machine. I want it back." => Predicted Keyword: "inquire"
Text: "Investing in stocks is risky." => Predicted Keyword: "inquire"
Text: "Why are your fixed deposit rates so terrible? I don't want to roll over my fixed deposit." => Predicted Keyword: "inquire"
